<a href="https://colab.research.google.com/github/jordipozo/CEIABD/blob/main/Fine_tuning_de_un_modelo_pre_entrenado_con_Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***NOTA IMPORTANTE***.

Es necesario ejecutar la instalación de 'accelerate' y, a posteriori, reiniciar la sesión *(Entorno de ejecución -> Reiniciar sesión)*

In [ ]:
!pip install accelerate -U

# FINE TUNNING en modelos pre-entrenados con Hugging Face

El uso de un modelo pre-entrenado tiene importantes ventajas. Reduce los costos de computación, la huella de carbono, y te permite utilizar modelos de última generación sin tener que entrenar uno desde cero. 🤗 Transformers proporciona acceso a miles de modelos pre-entrenados en una amplia gama de tareas. Cuando se utiliza un modelo pre-entrenado, este se entrena con un dataset específico para una determinada tarea.   

Esto se conoce como **fine-tuning**, una técnica de entrenamiento increíblemente poderosa.   

En este notebook se va a realizar un proceso de **fine-tuning** a un modelo pre-entrenado con un framework de Deep Learning de un modelo pre-entrenado con 🤗 Transformers Trainer.

In [1]:
!git config --global credential.helper store

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).

### Preparación de un dataset

Antes de aplicar fine-tuning a un modelo pre-entrenado, descargamos un dataset y lo preparamos para el entrenamiento.

Cargaremos el dataset de *Yelp Reviews*:

In [5]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]
{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. I\'ve worked at more than one location. I expect bad days, bad moods, and the occasional mistake. But I have yet to have a decent experience at this store. It will remain a place I avoid unless someone in my party needs to avoid illness from low blood sugar. Perhaps I should go back to the racially biased service of Steak n Shake instead!'}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

Se necesita un tokenizador para procesar el texto e incluir una estrategia para el padding y el truncamiento, para manejar cualquier longitud de secuencia variable.   
Para procesar el dataset en un solo paso, se utiliza el método de Datasets **map** para aplicar una función de preprocesamiento sobre todo el dataset:

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Si se desea, se puede crear un subconjunto más pequeño del dataset completo para aplicarle fine-tuning y así reducir el tiempo.
```
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
```

In [13]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

## Fine Tunning con 'Trainer'

Transformers proporciona una clase **Trainer** optimizada para el entrenamiento de modelos de Transformers, haciendo más fácil el inicio del entrenamiento sin necesidad de escribir manualmente nuestro propio ciclo.   
La API del **Trainer** soporta una amplia gama de opciones de entrenamiento y características como el *logging*, el *gradient accumulation* y el *mixed precision*.

Se empieza cargando el modelo y especificando el número de labels previstas. A partir del Card Dataset de Yelp Review, que sabemos que tiene 5 labels:

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NOTA: aparece una advertencia acerca de que algunos de los pesos pre-entrenados que no están siendo utilizados y que algunos pesos están siendo inicializados al azar.   
Esto es completamente normal. El **head/cabezal** pre-entrenado del modelo BERT se descarta y se sustituye por un **head** de clasificación inicializado aleatoriamente.   
Se puede aplicar *fine-tuning* a este nuevo **head** del modelo en tu tarea de clasificación de secuencias haciendo transfer learning del modelo pre-entrenado.

### Hiperparámetros de entrenamiento
A continuación, se crea una clase TrainingArguments que contenga todos los hiperparámetros que se pueden ajustar así como los indicadores para activar las diferentes opciones de entrenamiento.   
Para este tutorial es posible empezar con los hiperparámetros de entrenamiento por defecto.

En Colab, realizaremos lo siguiente:

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Se especifica, a continuación, dónde se van a guardar los checkpoints del entrenamiento:
(En el ejemplo se usa la ruta de Google Drive)

In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="/content/drive/MyDrive/test_trainer")

## Métricas.
El **Trainer** no evalúa automáticamente el rendimiento del modelo durante el entrenamiento. Es necesario pasarle a **Trainer** una función para calcular y hacer un reporte de las métricas. La librería de Datasets proporciona una función de *accuracy* simple que se puede cargar con la función **load_metric**:

In [8]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

<ipython-input-8-56b0b4182bac>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Se define la función compute en metric para calcular el accuracy de las predicciones. Antes de pasar las predicciones a compute, es necesario convertir las predicciones a logits (todos los modelos de Transformers devuelven logits).

In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Si se quiere controlar las métricas de evaluación durante el fine-tuning, se debe especificar el parámetro ***evaluation_strategy*** en los argumentos de entrenamiento para que el modelo tenga en cuenta la métrica de evaluación al final de cada época:

In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## **Trainer**
Se crea un objeto Trainer con el modelo, argumentos de entrenamiento, conjuntos de datos de entrenamiento y de prueba, y la función de evaluación:

In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

Realizamos el entrenamiento:

In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.621689,0.245000
2,No log,1.413915,0.399000
3,No log,1.209548,0.475000


TrainOutput(global_step=375, training_loss=1.44230859375, metrics={'train_runtime': 357.0972, 'train_samples_per_second': 8.401, 'train_steps_per_second': 1.05, 'total_flos': 789354427392000.0, 'train_loss': 1.44230859375, 'epoch': 3.0})